# Summary and Practical Guidelines

## Objective

Synthesize key findings and provide practical guidelines for numerical algorithm selection.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

plt.rcParams["figure.figsize"] = (12, 6)

## Key Empirical Findings

### 1. Summation

| Algorithm | Error Growth | When to Use |
|-----------|--------------|-------------|
| Naive | O(nε) | Small n, non-critical |
| Kahan | O(ε) + O(nε²) | Critical applications |
| Pairwise | O(log n · ε) | Good balance |

**Recommendation**: Use Kahan for critical sums, especially when n > 10⁶

### 2. Polynomial Evaluation

| Method | Stability | Cost |
|--------|-----------|------|
| Naive | Unstable | O(n²) ops |
| Horner | Stable | O(n) ops |

**Recommendation**: Always use Horner method

### 3. Linear Systems

| Method | Condition Number Effect | When to Use |
|--------|------------------------|-------------|
| GE without pivoting | Unstable | Never |
| GE with pivoting | κ(A) | General systems |
| Normal equations | κ(A)² | Never for ill-conditioned |
| QR factorization | κ(A) | Least squares, κ > 10⁶ |
| Matrix inversion | Unstable + slow | Never |

**Recommendation**: Always pivot; use QR for ill-conditioned least squares

## Practical Decision Tree

```
Problem: Solve Ax = b
│
├─ Square system (m = n)?
│  ├─ Yes → Use Gaussian elimination with pivoting
│  └─ No → Least squares problem
│     ├─ κ(A) < 10⁶ → Normal equations acceptable
│     └─ κ(A) ≥ 10⁶ → Use QR factorization
│
├─ Need matrix inverse?
│  ├─ For solving systems → NO! Use direct solve
│  └─ Actually need A⁻¹ → OK, but rare
│
└─ Polynomial evaluation?
   └─ Always use Horner method
```

In [ ]:
# Create comparison table
data = {
    "Task": [
        "Summation (n < 1000)",
        "Summation (n > 10⁶)",
        "Polynomial eval",
        "Linear system (well-cond)",
        "Linear system (ill-cond)",
        "Least squares (well-cond)",
        "Least squares (ill-cond)",
        "Matrix inversion"
    ],
    "Recommended Algorithm": [
        "Naive or NumPy sum",
        "Kahan summation",
        "Horner method",
        "GE with pivoting",
        "GE with pivoting (expect large error)",
        "Normal equations or QR",
        "QR factorization",
        "Direct solve instead!"
    ],
    "Expected Error": [
        "O(nε)",
        "O(ε)",
        "O(nε)",
        "O(κε)",
        "O(κε) - large!",
        "O(κ²ε) or O(κε)",
        "O(κε)",
        "Worse than solve"
    ]
}

df = pd.DataFrame(data)
print("\nPractical Algorithm Selection Guide")
print("=" * 80)
print(df.to_string(index=False))

## Fundamental Principles

### 1. Conditioning vs Stability

- **Conditioning**: Property of the problem (cannot be changed)
- **Stability**: Property of the algorithm (can be improved)
- **Key insight**: Stable algorithm + ill-conditioned problem = large error (unavoidable)

### 2. Backward Stability

- **Definition**: Algorithm produces exact answer to nearby problem
- **Gold standard**: Backward error ≈ ε
- **Implication**: Forward error ≤ κ(A) × ε

### 3. Error Bounds

- **Achievable accuracy**: ≈ κ(A) × ε
- **When κ(A) > 1/ε ≈ 10¹⁶**: Problem is essentially singular
- **Practical limit**: κ(A) < 10¹⁰ for reliable results

### 4. Catastrophic Cancellation

- **Cause**: Subtracting nearly equal numbers
- **Effect**: Loss of significant digits
- **Solution**: Algorithmic reformulation (e.g., log1p, expm1)

### 5. Never Explicitly Invert

- **Reason 1**: More expensive (O(n³) vs O(n³) but larger constant)
- **Reason 2**: Less accurate (amplifies errors)
- **Exception**: When you actually need A⁻¹ itself (rare)

## When to Worry

### Red Flags

1. **Condition number > 10¹⁰**: Near machine precision limit
2. **Subtracting similar quantities**: Risk of cancellation
3. **Forming A^T A**: Squares condition number
4. **No pivoting**: Algorithm may fail
5. **Explicit inversion**: Better alternatives exist

### Diagnostic Workflow

1. **Check condition number**: `np.linalg.cond(A)`
2. **Compute residual**: `||Ax - b||`
3. **Estimate backward error**: `||r|| / (||A|| ||x|| + ||b||)`
4. **Compare to expected**: Should be ≈ ε for stable algorithms
5. **If too large**: Suspect algorithmic instability

In [ ]:
# Summary visualization
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Error growth comparison
ax = axes[0, 0]
n_values = np.logspace(1, 7, 50)
eps = np.finfo(np.float64).eps
ax.loglog(n_values, n_values * eps, label="Naive sum: O(nε)", linewidth=2)
ax.loglog(n_values, np.ones_like(n_values) * eps, label="Kahan sum: O(ε)", linewidth=2)
ax.loglog(n_values, np.log2(n_values) * eps, label="Pairwise: O(log n · ε)", linewidth=2)
ax.set_xlabel("Problem size (n)")
ax.set_ylabel("Expected error")
ax.set_title("Summation Error Growth")
ax.legend()
ax.grid(True, alpha=0.3)

# 2. Conditioning effect
ax = axes[0, 1]
kappa_values = np.logspace(0, 16, 100)
ax.loglog(kappa_values, kappa_values * eps, label="Stable algorithm", linewidth=2)
ax.loglog(kappa_values, kappa_values**2 * eps, label="Normal equations", linewidth=2, linestyle="--")
ax.axhline(1.0, color="k", linestyle=":", alpha=0.5, label="Total loss of accuracy")
ax.axvline(1/eps, color="r", linestyle=":", alpha=0.5, label="Singular limit")
ax.set_xlabel("Condition number κ(A)")
ax.set_ylabel("Expected relative error")
ax.set_title("Conditioning Effect on Error")
ax.legend()
ax.grid(True, alpha=0.3)

# 3. Algorithm comparison
ax = axes[1, 0]
algorithms = ["Naive\nsum", "Kahan\nsum", "Naive\npoly", "Horner", "GE\nno pivot", "GE\npivot", "Normal\neq", "QR"]
stability = [2, 9, 3, 9, 1, 9, 5, 9]  # Subjective rating
colors = ["red" if s < 5 else "orange" if s < 8 else "green" for s in stability]
ax.barh(algorithms, stability, color=colors, alpha=0.7)
ax.set_xlabel("Stability Rating")
ax.set_title("Algorithm Stability Comparison")
ax.set_xlim(0, 10)
ax.grid(True, alpha=0.3, axis="x")

# 4. Decision regions
ax = axes[1, 1]
kappa_plot = np.logspace(0, 12, 100)
ax.fill_between(kappa_plot, 0, 1, where=(kappa_plot < 1e6), 
                alpha=0.3, color="green", label="Well-conditioned")
ax.fill_between(kappa_plot, 0, 1, where=((kappa_plot >= 1e6) & (kappa_plot < 1e10)), 
                alpha=0.3, color="orange", label="Ill-conditioned")
ax.fill_between(kappa_plot, 0, 1, where=(kappa_plot >= 1e10), 
                alpha=0.3, color="red", label="Near-singular")
ax.set_xscale("log")
ax.set_xlabel("Condition number κ(A)")
ax.set_yticks([])
ax.set_title("Problem Conditioning Regions")
ax.legend(loc="upper left")
ax.set_xlim(1, 1e12)

plt.tight_layout()
plt.savefig("../plots/06_summary_visualization.png", dpi=150, bbox_inches="tight")
plt.show()

## Final Recommendations

### For Practitioners

1. **Always check condition numbers** before solving
2. **Use backward stable algorithms** (pivoting, QR, Kahan)
3. **Never invert matrices** for solving systems
4. **Reformulate** to avoid catastrophic cancellation
5. **Expect error ≈ κ(A) × ε** for stable algorithms

### For Algorithm Designers

1. **Aim for backward stability** (gold standard)
2. **Minimize operation count** (fewer ops = less error)
3. **Avoid forming A^T A** (squares condition number)
4. **Use pivoting/equilibration** for robustness
5. **Test on ill-conditioned problems** (reveals instabilities)

### For Researchers

1. **Distinguish conditioning from stability**
2. **Report both forward and backward errors**
3. **Test across range of condition numbers**
4. **Compare to theoretical error bounds**
5. **Document numerical properties** of algorithms